In [1]:
from __future__ import print_function

import re
import sys
from operator import add

In [2]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

In [3]:
def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [4]:
lines = spark.read.text("../data/graphx/followers.txt").rdd.map(lambda r: r[0])

In [5]:
links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()

In [6]:
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [7]:
for iteration in range(10):
        # Calculates URL contributions to the rank of other URLs.
        contribs = links.join(ranks).flatMap(
            lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))

In [9]:
ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

# Collects all URL ranks and dump them to console.
for (link, rank) in ranks.collect():
    print("%s has rank: %s." % (link, rank))


1 has rank: 1.8499999999999999.
2 has rank: 1.0.
3 has rank: 1.0.
7 has rank: 1.4249999999999998.
6 has rank: 0.575.
